In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType
import sys

CATALOGO = "catalogo_energia"
SCHEMA_SILVER = "silver"
SCHEMA_GOLD = "gold"
try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("modelagem_gold_databricks").getOrCreate()

print('Spark disponível:', spark is not None)
print('Versão do Spark:', spark.version)

### Leitura da tabela Silver

In [0]:

silver_table = f"{CATALOGO}.{SCHEMA_SILVER}.energia_consumo_silver"
print('Tentando ler a tabela:', silver_table)

df_silver = spark.read.table(silver_table)
print(f"Total de linhas lidas da Silver: {df_silver.count()}")

display(df_silver.limit(5))
print('\nSchema:')
df_silver.printSchema()

### Validação de colunas requeridas

In [0]:
required_cols = [
    'regiao', 'mes_de_referencia', 'consumo_em_kwh', 'valor_da_conta',
    'tipo_de_cliente', 'cidade', 'bairro', 'tipo_de_consumo', 'valor_de_imposto'
]
missing = [c for c in required_cols if c not in df_silver.columns]

if missing:
    msg = f"Colunas faltando no dataframe silver: {missing}"
    print(msg)
    # Levanta um erro para interromper execução segura
    raise ValueError(msg)
else:
    print('Todas as colunas requeridas estão presentes.')

### US-3.1: Agregação Mensal Regional Total (Fato para Picos de Demanda)

In [0]:
try:
    df_total_regional = df_silver.groupBy(
        'regiao', 'mes_de_referencia'
    ).agg(
        F.sum('consumo_em_kwh').alias('consumo_total_kwh'),
        F.sum('valor_da_conta').alias('custo_total'),
        F.count('*').alias('total_registros')
    )

    table_consumo_mensal_regional = f"{CATALOGO}.{SCHEMA_GOLD}.ft_consumo_mensal_regional"
    df_total_regional.write.format('delta') \
        .mode('overwrite') \
        .option('overwriteSchema', 'true') \
        .saveAsTable(table_consumo_mensal_regional)

    print(f'Tabela Fato {table_consumo_mensal_regional} persistida com sucesso.')
    display(df_total_regional.limit(5))
except Exception as e:
    print('Erro na US-3.1:', e)
    raise

### US-3.2: Agregação por Tipo de Cliente (Fato para Custo/Rentabilidade)

In [0]:
try:
    df_custo_segmento = df_silver.groupBy(
        'tipo_de_cliente'
    ).agg(
        F.sum('consumo_em_kwh').alias('consumo_total_kwh'),
        F.sum('valor_da_conta').alias('custo_total'),
        (F.sum('valor_da_conta') / F.sum('consumo_em_kwh')).alias('custo_medio_por_kwh_segmento')
    )

    table_custo_segmento = f"{CATALOGO}.{SCHEMA_GOLD}.ft_custo_segmento"
    df_custo_segmento.write.format('delta') \
        .mode('overwrite') \
        .option('overwriteSchema', 'true') \
        .saveAsTable(table_custo_segmento)

    print(f'Tabela Fato {table_custo_segmento} persistida com sucesso.')
    display(df_custo_segmento.limit(5))
except Exception as e:
    print('Erro na US-3.2:', e)
    raise

Databricks visualization. Run in Databricks to view.

### US-3.4: Criar a Consumo por cidade

In [0]:
try:
    # US-Prefeito: Consumo e Custo agregado por Cidade e Mês
    # Esta tabela Fato permite a comparação do consumo entre diferentes cidades.
    df_consumo_cidade = df_silver.groupBy(
        'cidade', 'mes_de_referencia'
    ).agg(
        F.sum('consumo_em_kwh').alias('consumo_total_kwh'),
        F.sum('valor_da_conta').alias('custo_total'),
        F.count('*').alias('total_registros')
    )

    table_consumo_cidade = f"{CATALOGO}.{SCHEMA_GOLD}.ft_consumo_mensal_cidade"
    df_consumo_cidade.write.format('delta') \
        .mode('overwrite') \
        .option('overwriteSchema', 'true') \
        .saveAsTable(table_consumo_cidade)

    print(f'Tabela Fato {table_consumo_cidade} persistida com sucesso para comparativo entre cidades.')
    display(df_consumo_cidade.limit(5))
except Exception as e:
    print('Erro na US-Prefeito (ft_consumo_mensal_cidade):', e)
    raise

Databricks visualization. Run in Databricks to view.

### US-3.3: Criar a Dimensão D-Localidade

In [0]:

try:
    df_localidade_dim = df_silver.select('regiao', 'cidade', 'bairro').distinct()
    df_localidade_dim = df_localidade_dim.withColumn(
        'localidade_sk', F.monotonically_increasing_id().cast(LongType())
    )
    table_localidade = f"{CATALOGO}.{SCHEMA_GOLD}.d_localidade"
    df_localidade_dim.write.format('delta') \
        .mode('overwrite') \
        .option('overwriteSchema', 'true') \
        .saveAsTable(table_localidade)

    print(f'Dimensão {table_localidade} persistida com sucesso.')
    display(df_localidade_dim.limit(5))
except Exception as e:
    print('Erro na US-3.3:', e)
    raise

### US-5: Agregação Mensal Regional Comparativa (Fato)

In [0]:
try:
    df_consumo_agregado = df_silver.groupBy(
        'regiao', 'mes_de_referencia', 'tipo_de_consumo'
    ).agg(
        F.sum('consumo_em_kwh').alias('consumo_total_kwh'),
        F.sum('valor_da_conta').alias('custo_total'),
        F.sum('valor_de_imposto').alias('imposto_total'),
        (F.sum('valor_da_conta') / F.sum('consumo_em_kwh')).alias('custo_medio_por_kwh')
    )
    table_consumo_mensal_comparativo = f"{CATALOGO}.{SCHEMA_GOLD}.ft_consumo_mensal_comparativo"
    df_consumo_agregado.write.format('delta') \
        .mode('overwrite') \
        .option('overwriteSchema', 'true') \
        .saveAsTable(table_consumo_mensal_comparativo)

    print(f'Tabela Fato {table_consumo_mensal_comparativo} persistida com sucesso.')
    display(df_consumo_agregado.limit(5))
except Exception as e:
    print('Erro na US-5:', e)

### Checagens pós-persistência (confere existência das tabelas salvas)

In [0]:
tables_to_check = [
    f"{CATALOGO}.{SCHEMA_GOLD}.d_localidade",
    f"{CATALOGO}.{SCHEMA_GOLD}.ft_consumo_mensal_comparativo",
    f"{CATALOGO}.{SCHEMA_GOLD}.ft_consumo_mensal_regional",
    f"{CATALOGO}.{SCHEMA_GOLD}.ft_custo_segmento",
    f"{CATALOGO}.{SCHEMA_GOLD}.ft_consumo_mensal_cidade"
]

for t in tables_to_check:
    # CORREÇÃO: Usamos a API pública do Spark (spark.catalog) em vez da interna (_jsparkSession).
    # Passamos 't' direto, pois ele já contém o caminho completo (ex: cat.schema.tabela)
    exists = spark.catalog.tableExists(t)
    print(f"Tabela {t} -> existe? {exists}")

### 03 - Agregação da Camada GOLD (Modelagem Dimensional)

#### Objetivo
Criar o Modelo de Dados Dimensional (Tabelas FATO e DIMENSÃO) a partir dos dados limpos da Camada Silver, visando otimizar a performance de consultas e responder às User Stories de Negócio.

#### User Stories Concluídas
* **US-3.5:** Criação da Dimensão D-Localidade.
* **US-5:** Criação da Fato Comparativa (Geral vs. Limpa).
* **US-3.1:** Criação da Fato Total Regional (Picos de Demanda).
* **US-3.2:** Criação da Fato por Tipo de Cliente (Custo Médio).


#### Tabelas Finais Persistidas
1.  **Dimensão:** `d_localidade` (Região, Cidade, Bairro, Localidade_SK)
2.  **Fato 1 (Comparativa):** `ft_consumo_mensal_comparativo` (Agregada por Região, Mês, Tipo de Consumo)
3.  **Fato 2 (Total):** `ft_consumo_mensal_regional` (Agregada por Região, Mês)
4.  **Fato 3 (Segmento):** `ft_custo_segmento` (Agregada por Tipo de Cliente)